[View in Colaboratory](https://colab.research.google.com/github/zhh210/flea_market/blob/master/crf_tagging.ipynb)

In [9]:
# !git clone https://github.com/epwalsh/pytorch-crf.git
# !git clone https://github.com/kamalkraj/Named-Entity-Recognition-with-Bidirectional-LSTM-CNNs.git
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip
# !pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
# !pip3 install torchvision
# !pip3 install -r pytorch-crf/requirements.txt
!ls

glove.6B.100d.txt				       pytorch-crf
glove.6B.200d.txt				       sample_data
glove.6B.300d.txt				       test.txt
glove.6B.50d.txt				       train.txt
glove.6B.zip					       valid.txt
Named-Entity-Recognition-with-Bidirectional-LSTM-CNNs


In [0]:
import pandas as pd
import numpy as np
def process_data(fname=None,save_as=None):
    df = pd.read_csv(fname,sep=' ',skip_blank_lines=False)
    df.drop(df.index[0], inplace=True)
    
    cols = list(df.columns)
    df.drop(list(df[df[cols[0]]==' '].index),inplace=True)
    df = df[[cols[0],cols[-1]]].replace(' ',np.nan).replace('\t',np.nan).fillna(np.nan)
    with open(save_as,'w') as f:
        for id, it in df.iterrows():
            if not pd.isnull(it[cols[0]]):
                f.write('\t'.join([str(i) for i in it])+'\n')
            else:
                f.write('\n')
#     df.to_csv(save_as, index=None,header=None,sep='\t')
process_data('Named-Entity-Recognition-with-Bidirectional-LSTM-CNNs/data/valid.txt',save_as='valid.txt')
process_data('Named-Entity-Recognition-with-Bidirectional-LSTM-CNNs/data/train.txt',save_as='train.txt')
process_data('Named-Entity-Recognition-with-Bidirectional-LSTM-CNNs/data/test.txt',save_as='test.txt')

In [110]:
# !cat -n train.txt | tail -100
import pandas as pd
import numpy as np
a = pd.read_csv('test.txt',skip_blank_lines=False,header=None,sep='\t')
# a = a.replace(' ',np.nan)
print(a.loc[(a[0].shift() == a[0]) & (pd.isnull(a[0]))].index)
# a.replace(' ').fillna('***').loc[[77,78,79,80]]
# a.replace(' ',np.nan).loc[[77,78,79]]
print(a.loc[[7005,7006,8572]])
# a.head()
# list(a[a[0]==' '].index)
a.tail()

Int64Index([], dtype='int64')
                    0  1
7005  characteristics  O
7006              and  O
8572              7-1  O


,0,1
49922,brother,O
49923,",",O
49924,Bobby,B-PER
49925,.,O
49926,NaN,NaN


In [112]:
!head -n20 train.txt test.txt pytorch-crf/test/data/sample_dataset.txt

==> train.txt <==
EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
lamb	O
.	O

Peter	B-PER
Blackburn	I-PER

BRUSSELS	B-LOC
1996-08-22	O

The	O
European	B-ORG
Commission	I-ORG
said	O

==> test.txt <==
SOCCER	O
-	O
JAPAN	B-LOC
GET	O
LUCKY	O
WIN	O
,	O
CHINA	B-PER
IN	O
SURPRISE	O
DEFEAT	O
.	O

Nadim	B-PER
Ladki	I-PER

AL-AIN	B-LOC
,	O
United	B-LOC
Arab	I-LOC

==> pytorch-crf/test/data/sample_dataset.txt <==
hi	O
there	O

my	O
name	O
is	O
Pete	B-NAME

I'	O
m	O
Evan	B-NAME
Walsh	I-NAME

hello	O



In [0]:
import sys; sys.path.append("./pytorch-crf")
# import pycrf
from pycrf.train import Learner


In [63]:
!ls *.txt -l

-rw-rw-r-- 1 root root  347116733 Aug  4  2014 glove.6B.100d.txt
-rw-rw-r-- 1 root root  693432828 Aug  4  2014 glove.6B.200d.txt
-rw-rw-r-- 1 root root 1037962819 Aug 27  2014 glove.6B.300d.txt
-rw-rw-r-- 1 root root  171350079 Aug  4  2014 glove.6B.50d.txt
-rw-r--r-- 1 root root     403553 Oct 26 04:22 test.txt
-rw-r--r-- 1 root root    1765134 Oct 26 04:22 train.txt
-rw-r--r-- 1 root root     441905 Oct 26 04:21 valid.txt


In [0]:
learner = Learner.build(train="test.txt",
                        validation="valid.txt",
                        word_vectors="glove.6B.50d.txt",
                        verbose=True,
                        batch_size=100,
                        cuda=True)
learner.fit()



Loading pretrained word vectors from glove.6B.50d.txt
Loading datasets
Loading file test.txt
Loaded 3683 sentences for training
Loading file valid.txt
Loaded 3468 sentences for validation
Found the following labels: ['O', 'B-LOC', 'B-PER', 'I-PER', 'I-LOC', 'B-MISC', 'I-MISC', 'B-ORG', 'I-ORG']
LSTMCRF(
  (char_feats_layer): CharLSTM(
    (char_embedding): CharEmbedding(
      (embedding): Embedding(96, 50, padding_idx=0)
    )
    (rnn): LSTM(50, 50, batch_first=True, bidirectional=True)
  )
  (word_embedding): Embedding(400000, 50)
  (rnn): LSTM(150, 50, batch_first=True, bidirectional=True)
  (rnn_to_crf): Linear(in_features=100, out_features=9, bias=True)
  (crf): ConditionalRandomField()
)






1:   0%|          | 0/3683 [00:00<?, ?it/s]



1:   3%|▎         | 100/3683 [00:02<01:44, 34.27it/s]



1:   3%|▎         | 100/3683 [00:02<01:44, 34.27it/s, loss=3430 (3430), lr=0.01]



1:   5%|▌         | 200/3683 [00:08<02:07, 27.23it/s, loss=3430 (3430), lr=0.01]



1:   5%|▌         | 200/3683 [00:08<02:07, 27.23it/s, loss=1.141e+04 (7980), lr=0.01]



1:   8%|▊         | 300/3683 [00:13<02:20, 24.12it/s, loss=1.141e+04 (7980), lr=0.01]



1:   8%|▊         | 300/3683 [00:13<02:20, 24.12it/s, loss=4.251e+04 (3.11e+04), lr=0.01]



1:  11%|█         | 400/3683 [00:18<02:27, 22.19it/s, loss=4.251e+04 (3.11e+04), lr=0.01]



1:  11%|█         | 400/3683 [00:18<02:27, 22.19it/s, loss=5.895e+04 (1.643e+04), lr=0.01]



1:  14%|█▎        | 500/3683 [00:24<02:32, 20.94it/s, loss=5.895e+04 (1.643e+04), lr=0.01]



1:  14%|█▎        | 500/3683 [00:24<02:32, 20.94it/s, loss=7.224e+04 (1.329e+04), lr=0.01]



1:  16%|█▋        | 600/3683 [00:29<02:34, 19.99it/s, loss=7.224e+04 (1.329e+0

Validation set scores:
precision:      0.5213
recall:         0.4860
f1:             0.5030
token accuracy: 0.9056







2:   0%|          | 0/3683 [00:00<?, ?it/s]



2:   3%|▎         | 100/3683 [00:02<01:25, 42.12it/s]



2:   3%|▎         | 100/3683 [00:02<01:25, 42.12it/s, loss=299.8 (299.8), lr=0.01]



2:   5%|▌         | 200/3683 [00:07<01:48, 32.15it/s, loss=299.8 (299.8), lr=0.01]



2:   5%|▌         | 200/3683 [00:07<01:48, 32.15it/s, loss=604.2 (304.4), lr=0.01]



2:   8%|▊         | 300/3683 [00:12<02:03, 27.43it/s, loss=604.2 (304.4), lr=0.01]



2:   8%|▊         | 300/3683 [00:12<02:03, 27.43it/s, loss=963.4 (359.2), lr=0.01]



2:  11%|█         | 400/3683 [00:17<02:13, 24.64it/s, loss=963.4 (359.2), lr=0.01]



2:  11%|█         | 400/3683 [00:17<02:13, 24.64it/s, loss=1374 (411.1), lr=0.01] 



2:  14%|█▎        | 500/3683 [00:22<02:18, 22.91it/s, loss=1374 (411.1), lr=0.01]



2:  14%|█▎        | 500/3683 [00:22<02:18, 22.91it/s, loss=1926 (551.8), lr=0.01]



2:  16%|█▋        | 600/3683 [00:27<02:27, 20.96it/s, loss=1926 (551.8), lr=0.01]



2:  16%|█▋        | 600/3683 [00:27

Validation set scores:
precision:      0.6049
recall:         0.6432
f1:             0.6235
token accuracy: 0.9210



...





7:   0%|          | 0/3683 [00:00<?, ?it/s]



7:   3%|▎         | 100/3683 [00:02<01:39, 35.83it/s]



7:   3%|▎         | 100/3683 [00:02<01:39, 35.83it/s, loss=132.9 (132.9), lr=0.01]



7:   5%|▌         | 200/3683 [00:08<02:03, 28.21it/s, loss=132.9 (132.9), lr=0.01]



7:   5%|▌         | 200/3683 [00:08<02:03, 28.21it/s, loss=240.1 (107.2), lr=0.01]



7:   8%|▊         | 300/3683 [00:13<02:14, 25.19it/s, loss=240.1 (107.2), lr=0.01]



7:   8%|▊         | 300/3683 [00:13<02:14, 25.19it/s, loss=329.8 (89.75), lr=0.01]



7:  11%|█         | 400/3683 [00:18<02:21, 23.26it/s, loss=329.8 (89.75), lr=0.01]



7:  11%|█         | 400/3683 [00:18<02:21, 23.26it/s, loss=419.9 (90.06), lr=0.01]



7:  14%|█▎        | 500/3683 [00:23<02:26, 21.73it/s, loss=419.9 (90.06), lr=0.01]



7:  14%|█▎        | 500/3683 [00:23<02:26, 21.73it/s, loss=521.6 (101.7), lr=0.01]



7:  16%|█▋        | 600/3683 [00:29<02:30, 20.42it/s, loss=521.6 (101.7), lr=0.01]



7:  16%|█▋        | 600/3683 [00

Validation set scores:
precision:      0.8063
recall:         0.7967
f1:             0.8015
token accuracy: 0.9591







8:   0%|          | 0/3683 [00:00<?, ?it/s]



8:   3%|▎         | 100/3683 [00:02<01:32, 38.57it/s]



8:   3%|▎         | 100/3683 [00:02<01:32, 38.57it/s, loss=104.8 (104.8), lr=0.01]



8:   5%|▌         | 200/3683 [00:07<01:56, 29.92it/s, loss=104.8 (104.8), lr=0.01]



8:   5%|▌         | 200/3683 [00:07<01:56, 29.92it/s, loss=195.5 (90.72), lr=0.01]



8:   8%|▊         | 300/3683 [00:12<02:08, 26.39it/s, loss=195.5 (90.72), lr=0.01]



8:   8%|▊         | 300/3683 [00:12<02:08, 26.39it/s, loss=271.2 (75.68), lr=0.01]



8:  11%|█         | 400/3683 [00:17<02:18, 23.64it/s, loss=271.2 (75.68), lr=0.01]



8:  11%|█         | 400/3683 [00:17<02:18, 23.64it/s, loss=353.3 (82.16), lr=0.01]



8:  14%|█▎        | 500/3683 [00:23<02:26, 21.69it/s, loss=353.3 (82.16), lr=0.01]



8:  14%|█▎        | 500/3683 [00:23<02:26, 21.69it/s, loss=441.6 (88.28), lr=0.01]



8:  16%|█▋        | 600/3683 [00:28<02:27, 20.86it/s, loss=441.6 (88.28), lr=0.01]



8:  16%|█▋        | 600/3683 [00

Validation set scores:
precision:      0.8172
recall:         0.7991
f1:             0.8080
token accuracy: 0.9608







9:   0%|          | 0/3683 [00:00<?, ?it/s]



9:   3%|▎         | 100/3683 [00:02<01:30, 39.56it/s]



9:   3%|▎         | 100/3683 [00:02<01:30, 39.56it/s, loss=54.03 (54.03), lr=0.01]



9:   5%|▌         | 200/3683 [00:07<01:58, 29.35it/s, loss=54.03 (54.03), lr=0.01]



9:   5%|▌         | 200/3683 [00:07<01:58, 29.35it/s, loss=145.4 (91.32), lr=0.01]



9:   8%|▊         | 300/3683 [00:13<02:16, 24.78it/s, loss=145.4 (91.32), lr=0.01]



9:   8%|▊         | 300/3683 [00:13<02:16, 24.78it/s, loss=210.8 (65.42), lr=0.01]



9:  11%|█         | 400/3683 [00:18<02:21, 23.16it/s, loss=210.8 (65.42), lr=0.01]



9:  11%|█         | 400/3683 [00:18<02:21, 23.16it/s, loss=255.7 (44.96), lr=0.01]



9:  14%|█▎        | 500/3683 [00:23<02:23, 22.24it/s, loss=255.7 (44.96), lr=0.01]



9:  14%|█▎        | 500/3683 [00:23<02:23, 22.24it/s, loss=318.7 (62.92), lr=0.01]



9:  16%|█▋        | 600/3683 [00:28<02:25, 21.17it/s, loss=318.7 (62.92), lr=0.01]



9:  16%|█▋        | 600/3683 [00

In [120]:
dfs = []
for src, tgt in learner.dataset_valid:
    labs = [learner.vocab.labels_itos[i] for i in list(tgt.cpu().numpy())]
    preds = [learner.vocab.labels_itos[i] for i in learner.model.predict(*src)[0][0]]
    dfs.append(pd.DataFrame({'lab':labs,'pred':preds}))
    
df = pd.concat(dfs)
df

,lab,pred
0,O,O
1,O,O
2,B-ORG,B-ORG
3,O,I-ORG
4,O,O
5,O,O
6,O,O
7,O,O
8,O,O
9,O,O


In [124]:
pd.crosstab(df.lab, df.pred)

pred,B-LOC,B-MISC,B-ORG,B-PER,I-LOC,I-MISC,I-ORG,I-PER,O
lab,,,,,,,,,
B-LOC,1585,40,103,39,1,3,10,1,55
B-MISC,87,577,62,51,3,5,8,0,129
B-ORG,142,56,919,99,0,2,20,3,100
B-PER,23,8,5,1756,1,1,1,19,28
I-LOC,23,1,0,0,138,21,40,16,18
I-MISC,13,20,5,3,35,100,62,15,93
I-ORG,42,18,22,7,41,40,412,46,123
I-PER,0,0,0,19,1,3,2,1254,24
O,33,50,44,56,17,44,54,11,42026


In [115]:
! head -n79 train.txt > train_head.txt
for i, line in zip(range(100),open('train.txt','r').readlines()):
  print(i,line.encode('utf-8'))

0 b'EU\tB-ORG\n'
1 b'rejects\tO\n'
2 b'German\tB-MISC\n'
3 b'call\tO\n'
4 b'to\tO\n'
5 b'boycott\tO\n'
6 b'British\tB-MISC\n'
7 b'lamb\tO\n'
8 b'.\tO\n'
9 b'\n'
10 b'Peter\tB-PER\n'
11 b'Blackburn\tI-PER\n'
12 b'\n'
13 b'BRUSSELS\tB-LOC\n'
14 b'1996-08-22\tO\n'
15 b'\n'
16 b'The\tO\n'
17 b'European\tB-ORG\n'
18 b'Commission\tI-ORG\n'
19 b'said\tO\n'
20 b'on\tO\n'
21 b'Thursday\tO\n'
22 b'it\tO\n'
23 b'disagreed\tO\n'
24 b'with\tO\n'
25 b'German\tB-MISC\n'
26 b'advice\tO\n'
27 b'to\tO\n'
28 b'consumers\tO\n'
29 b'to\tO\n'
30 b'shun\tO\n'
31 b'British\tB-MISC\n'
32 b'lamb\tO\n'
33 b'until\tO\n'
34 b'scientists\tO\n'
35 b'determine\tO\n'
36 b'whether\tO\n'
37 b'mad\tO\n'
38 b'cow\tO\n'
39 b'disease\tO\n'
40 b'can\tO\n'
41 b'be\tO\n'
42 b'transmitted\tO\n'
43 b'to\tO\n'
44 b'sheep\tO\n'
45 b'.\tO\n'
46 b'\n'
47 b'Germany\tB-LOC\n'
48 b"'s\tO\n"
49 b'representative\tO\n'
50 b'to\tO\n'
51 b'the\tO\n'
52 b'European\tB-ORG\n'
53 b'Union\tI-ORG\n'
54 b"'s\tO\n"
55 b'veterinary\tO\n'
56 b'commit

In [47]:
for line in open('pytorch-crf/test/data/sample_dataset.txt','r').readlines():
  print(line.encode('utf-8'))

b'hi\tO\n'
b'there\tO\n'
b'\n'
b'my\tO\n'
b'name\tO\n'
b'is\tO\n'
b'Pete\tB-NAME\n'
b'\n'
b"I'\tO\n"
b'm\tO\n'
b'Evan\tB-NAME\n'
b'Walsh\tI-NAME\n'
b'\n'
b'hello\tO\n'
b'\n'


In [0]:
! head -n7 train.txt > train_head.txt

In [99]:
from pycrf.io import dataset
train = dataset.Dataset()
train.load_file('train_head.txt',learner.vocab)

Loading file train_head.txt


In [92]:
%debug

> /content/pytorch-crf/pycrf/nn/utils.py(73)sort_and_pad()
     71     """
     72     sorted_lens, sorted_idx = lengths.sort(0, descending=True)
---> 73     max_len = sorted_lens[0].item()
     74     padded = []
     75     for i in sorted_idx:

ipdb> u
> /content/pytorch-crf/pycrf/io/vocab.py(199)sent2tensor()
    197         # Words are sorted and padded by word length.
    198         word_tensors, lens, idxs = \
--> 199             sort_and_pad(word_tensors_list, word_lengths)
    200 
    201         # Send to device (like a cuda device) if device was given.

ipdb> 
> /content/pytorch-crf/pycrf/io/dataset.py(119)read_file()
    117                 else:
    118                     src.append(line_list[0])
--> 119                     tgt.append(line_list[1])
    120 
    121     def load_file(self,

ipdb> line
'\n'
ipdb> line_list
['']
ipdb> c


In [94]:
train.source


[]